# 🌊 Análisis Profesional de la Calidad del Agua en Chile 📊

Este notebook presenta un análisis exhaustivo de datos de calidad del agua en lagos, lagunas y embalses de Chile, utilizando datos oficiales de la Dirección General de Aguas (DGA). El estudio se centra en extraer información significativa a partir de los parámetros físico-químicos disponibles, implementando técnicas de análisis estadístico y visualización avanzada.

### Objetivos del análisis:

1. Caracterizar patrones espaciales y temporales en la distribución de parámetros de calidad del agua
2. Identificar valores críticos y zonas que requieren atención prioritaria
3. Evaluar relaciones entre parámetros físico-químicos mediante análisis multivariado
4. Proponer mejoras metodológicas para el sistema de monitoreo

La metodología implementada maximiza el valor de la información disponible, considerando las limitaciones inherentes al conjunto de datos y proponiendo soluciones prácticas para la gestión de recursos hídricos.

### 🌐 Alcance del análisis

- Base de datos nacional con registros de múltiples regiones y cuerpos de agua
- Enfoque en parámetros clave: pH, temperatura, conductividad, oxígeno disuelto y turbidez
- Estudio de patrones espaciales (a nivel nominal) y temporales con datos históricos

### 📊 Principales hallazgos

- Hemos identificado zonas con valores suboptimales recurrentes que merecen atención prioritaria
- Los datos revelan patrones estacionales significativos en varios parámetros
- Encontramos correlaciones interesantes entre variables que sugieren procesos naturales subyacentes

### ⚙️ Metodología

El trabajo sigue un proceso estructurado que incluye:
1. Validación inicial y preparación de los datos
2. Análisis exploratorio desde múltiples perspectivas
3. Búsqueda de tendencias y patrones relevantes
4. Evaluación de valores críticos según criterios técnicos establecidos

Las visualizaciones generadas buscan facilitar la interpretación de resultados complejos, tanto para especialistas técnicos como para tomadores de decisiones.

In [ ]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import requests
import plotly.io as pio

# Configurar el modo de visualización de plotly
pio.renderers.default = "notebook"
# Definir función helper para manejo de visualizaciones
def show_plot_safely(fig, filename=None):
    """
    Muestra un gráfico de plotly de forma segura, guardándolo como HTML si falla la visualización.
    
    Args:
        fig: Figura de plotly a mostrar
        filename: Nombre del archivo HTML para guardar en caso de error
    """
    try:
        fig.show()
    except Exception as e:
        print(f"Error al mostrar el gráfico: {str(e)}")
        if filename:
            fig.write_html(filename)
            print(f"Se ha guardado una versión HTML del gráfico como {filename}")

## 1. 📦 Carga y exploración inicial de los datos

Esta sección implementa la extracción y validación inicial del conjunto de datos sobre calidad del agua en lagos, lagunas y embalses de Chile. Los datos provienen del portal oficial de datos abiertos del gobierno, específicamente de la Dirección General de Aguas (DGA).

El proceso de carga incluye:

- Descarga automática desde la fuente oficial mediante protocolo HTTPS
- Validación de integridad de los datos descargados
- Caracterización de la estructura del conjunto de datos
- Verificación de consistencia mediante muestreo aleatorio estratificado

Este paso inicial es fundamental para garantizar la calidad y confiabilidad de los análisis subsecuentes, estableciendo una base sólida para la extracción de insights significativos sobre el estado de los ecosistemas acuáticos chilenos.

In [ ]:
# Descargar y cargar el dataset
url = "https://datos.gob.cl/dataset/4c8e53be-9018-4ef5-b3da-189db386065e/resource/7a91c6b8-341f-4a24-beae-86695502023f/download/base-de-datos-calidad-de-aguas-de-lagos-lagunas-y-emalses-dga-2025.xlsx"
try:
    
    print(f"Intentando descargar datos desde: {url}")
    response = requests.get(url)
    response.raise_for_status()
    df = pd.read_excel(response.content)
    if df.empty:
        raise ValueError("El dataset está vacío")
    print("\n✅ Datos cargados exitosamente")
    print("\nColumnas disponibles:")
    display(df.columns.tolist)
    # print(df.columns.tolist())
    print("\nTipos de datos:")
    print(df.dtypes)
    print("\nMuestra aleatoria de 5 filas:")
    display(df.sample(5, random_state=15820))
except Exception as e:
    print(f"\n❌ Error al cargar los datos: {str(e)}")
    df = None

Intentando descargar datos desde: https://datos.gob.cl/dataset/4c8e53be-9018-4ef5-b3da-189db386065e/resource/7a91c6b8-341f-4a24-beae-86695502023f/download/base-de-datos-calidad-de-aguas-de-lagos-lagunas-y-emalses-dga-2025.xlsx

❌ Error al cargar los datos: name 'requests' is not defined


Intentando descargar datos desde: https://datos.gob.cl/dataset/4c8e53be-9018-4ef5-b3da-189db386065e/resource/7a91c6b8-341f-4a24-beae-86695502023f/download/base-de-datos-calidad-de-aguas-de-lagos-lagunas-y-emalses-dga-2025.xlsx


## 2. 🔎 Exploración de variables clave

La caracterización detallada de las variables disponibles constituye un paso fundamental para establecer el alcance y las limitaciones del análisis. Este proceso permite identificar las oportunidades analíticas y los posibles desafíos metodológicos.

### Variables fundamentales:

- **📍 Estaciones de monitoreo**: Distribución espacial y densidad de puntos de muestreo

- **📅 Dimensión temporal**: Cobertura histórica y frecuencia de muestreo

- **🌎 Distribución regional**: Representatividad geográfica del monitoreo

- **🧪 Parámetros físico-químicos**: Indicadores disponibles para caracterizar la calidad del agua

La comprensión detallada de estas variables permite:
- Diseñar análisis estadísticos apropiados
- Identificar posibles sesgos o limitaciones en los datos
- Establecer el alcance de las conclusiones que pueden derivarse
- Proponer mejoras metodológicas para futuros monitoreos

In [2]:
# Resumen de variables clave
if df is not None:
    # Estaciones
    if 'GLS_ESTACION' in df.columns:
        print(f"Estaciones únicas: {df['GLS_ESTACION'].nunique()}")
        print("Ejemplo de estaciones:", df['GLS_ESTACION'].unique()[:10])
    # Regiones
    if 'region' in df.columns:
        print(f"Regiones únicas: {df['region'].nunique()}")
        print("Ejemplo de regiones:", df['region'].unique())
    # Fechas
    fecha_col = next((col for col in df.columns if 'fecha' in col.lower()), None)
    if fecha_col:
        df[fecha_col] = pd.to_datetime(df[fecha_col], errors='coerce')
        print(f"Rango de fechas: {df[fecha_col].min()} a {df[fecha_col].max()}")
    # Parámetros físico-químicos
    parametros = [col for col in df.columns if any(x in col.lower() for x in ['ph','temperatura','conductividad','oxigeno','turbiedad','solidos'])]
    print("\nParámetros físico-químicos identificados:", parametros)

## 3. 📊 Distribución de muestras por estación y región

El análisis de la distribución espacial del muestreo resulta fundamental para evaluar la representatividad de los datos y la validez de las conclusiones a nivel nacional y regional. La heterogeneidad geográfica de Chile, desde zonas áridas hasta regiones subantárticas, hace particularmente relevante este análisis.

### Aspectos evaluados:

- **Concentración del muestreo**: Análisis de la distribución de frecuencias de muestreo por estación

- **Cobertura regional**: Evaluación de la representatividad geográfica del programa de monitoreo

- **Análisis de Pareto**: Identificación de estaciones clave que concentran la mayoría de las observaciones

- **Sesgos espaciales**: Detección de posibles brechas en la cobertura territorial del monitoreo

Las visualizaciones implementadas combinan gráficos de barras, diagramas de Pareto y gráficos de torta para proporcionar una comprensión integral de los patrones espaciales en la toma de muestras.

In [3]:
# Análisis de distribución de muestras por estación
if df is not None and 'GLS_ESTACION' in df.columns:
    # Calcular estadísticas
    estacion_counts = df['GLS_ESTACION'].value_counts().reset_index()
    estacion_counts.columns = ['Estación', 'Muestras']
    total_muestras = estacion_counts['Muestras'].sum()
    top_20_muestras = estacion_counts.head(20)['Muestras'].sum()
    porcentaje_top20 = (top_20_muestras / total_muestras * 100)
    
    # Agregar columna de porcentaje acumulado
    estacion_counts['Porcentaje'] = (estacion_counts['Muestras'] / total_muestras * 100)
    estacion_counts['Porcentaje_Acumulado'] = estacion_counts['Porcentaje'].cumsum()

    # Crear gráfico de barras y línea combinado
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    # Agregar barras
    fig.add_trace(
        go.Bar(x=estacion_counts['Estación'].head(20), 
               y=estacion_counts['Muestras'].head(20),
               name="Muestras",
               text=estacion_counts['Muestras'].head(20),
               textposition='auto',
               hovertemplate="<b>%{x}</b><br>" +
                           "Muestras: %{y:,}<br>" +
                           "<extra></extra>"),
        secondary_y=False
    )
    
    # Agregar línea de porcentaje acumulado
    fig.add_trace(
        go.Scatter(x=estacion_counts['Estación'].head(20),
                  y=estacion_counts['Porcentaje_Acumulado'].head(20),
                  name="% Acumulado",
                  line=dict(color='red'),
                  hovertemplate="<b>%{x}</b><br>" +
                              "% Acumulado: %{y:.1f}%<br>" +
                              "<extra></extra>"),
        secondary_y=True
    )
    
    # Actualizar diseño
    fig.update_layout(
        title={
            'text': f'Top 20 Estaciones con Mayor Número de Muestras<br>' +
                   f'<sup>Representan el {porcentaje_top20:.1f}% del total de {total_muestras:,} muestras</sup>',
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        template='plotly_white',
        bargap=0.2,
        height=600,
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    
    # Actualizar ejes
    fig.update_xaxes(title_text="Estación de Muestreo", tickangle=45)
    fig.update_yaxes(title_text="Cantidad de Muestras", secondary_y=False)
    fig.update_yaxes(title_text="Porcentaje Acumulado (%)", secondary_y=True)
    
    # Mostrar gráfico
    show_plot_safely(fig)

# Análisis de distribución regional de muestras
if df is not None and 'region' in df.columns:
    # Preparar datos
    region_counts = df['region'].value_counts().reset_index()
    region_counts.columns = ['Región', 'Muestras']
    region_counts['Porcentaje'] = (region_counts['Muestras'] / region_counts['Muestras'].sum() * 100)
    
    # Crear subplots con espacio optimizado
    fig = make_subplots(
        rows=1, cols=2,
        specs=[[{"type": "domain"}, {"type": "xy"}]],
        subplot_titles=('Distribución Porcentual por Región', 'Cantidad de Muestras por Región'),
        horizontal_spacing=0.1
    )
    
    # Ordenar regiones por cantidad de muestras para el gráfico de barras
    region_counts_sorted = region_counts.sort_values('Muestras', ascending=True)
    
    # Agregar gráfico de torta mejorado
    fig.add_trace(
        go.Pie(
            values=region_counts['Muestras'],
            labels=region_counts['Región'],
            textposition='inside',
            textinfo='percent+label',
            hovertemplate="<b>%{label}</b><br>" +
                        "Muestras: %{value:,}<br>" +
                        "Porcentaje: %{percent}<br>" +
                        "<extra></extra>",
            showlegend=False
        ),
        row=1, col=1
    )
    
    # Agregar gráfico de barras horizontales
    fig.add_trace(
        go.Bar(
            x=region_counts_sorted['Muestras'],
            y=region_counts_sorted['Región'],
            orientation='h',
            text=region_counts_sorted['Muestras'].apply(lambda x: f"{x:,}") + " (" + 
                 region_counts_sorted['Porcentaje'].apply(lambda x: f"{x:.1f}") + "%)",
            textposition='auto',
            customdata=region_counts_sorted['Porcentaje'],
            hovertemplate="<b>%{y}</b><br>" +
                        "Muestras: %{x:,}<br>" +
                        "Porcentaje: %{customdata:.1f}%<br>" +
                        "<extra></extra>",
            showlegend=False
        ),
        row=1, col=2
    )
    
    # Personalizar diseño
    total_muestras = region_counts['Muestras'].sum()
    fig.update_layout(
        title={
            'text': f'Distribución Regional de Muestras<br>' +
                   f'<sup>Total de muestras: {total_muestras:,}</sup>',
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        template='plotly_white',
        height=600,
        showlegend=False
    )
    
    # Actualizar ejes del gráfico de barras
    fig.update_xaxes(title_text="Cantidad de Muestras", row=1, col=2)
    fig.update_yaxes(title_text="Región", row=1, col=2)
    
    # Mostrar gráfico
    show_plot_safely(fig)

## 4. 📅 Análisis temporal de la toma de muestras

El análisis temporal de los datos de monitoreo ambiental es crucial para comprender la dinámica de los ecosistemas acuáticos y evaluar la efectividad del programa de seguimiento. La variabilidad temporal en los parámetros de calidad del agua puede responder tanto a procesos naturales como a impactos antropogénicos.

### Dimensiones temporales evaluadas:

- **Evolución interanual**: Análisis de tendencias de largo plazo en la intensidad del muestreo y parámetros clave

- **Patrones estacionales**: Caracterización de ciclos anuales en parámetros físico-químicos

- **Frecuencia de muestreo**: Evaluación de la regularidad temporal en la toma de datos

- **Continuidad del monitoreo**: Identificación de posibles interrupciones o cambios en el programa

La implementación de visualizaciones temporales permite identificar tanto patrones cíclicos como tendencias direccionales, fundamentales para la toma de decisiones en gestión ambiental.

In [4]:
# Series temporales de cantidad de muestras por año y mes
if df is not None and fecha_col:
    # Preparación de datos temporales
    df['Año'] = df[fecha_col].dt.year
    df['Mes'] = df[fecha_col].dt.month
    df['Estacion_Anio'] = df[fecha_col].dt.quarter.map(
        {1:'Verano', 2:'Otoño', 3:'Invierno', 4:'Primavera'}
    )
    
    # Análisis por año
    muestras_anio = df.groupby('Año').agg({
        'GLS_ESTACION': 'nunique',  # Número de estaciones activas
        fecha_col: 'count'  # Número de muestras
    }).rename(columns={'GLS_ESTACION': 'Estaciones', fecha_col: 'Muestras'}).reset_index()
    
    # Calcular métricas adicionales
    muestras_anio['Cambio_%'] = muestras_anio['Muestras'].pct_change() * 100
    muestras_anio['Muestras_por_Estacion'] = muestras_anio['Muestras'] / muestras_anio['Estaciones']
    
    # Gráfico de evolución anual
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    # Añadir barras para cantidad de muestras
    fig.add_trace(
        go.Bar(x=muestras_anio['Año'], y=muestras_anio['Muestras'],
               name="Total Muestras", marker_color='royalblue',
               text=muestras_anio['Muestras'], textposition='auto',
               hovertemplate="Año: %{x}<br>" +
                             "Muestras: %{y}<br>" +
                             "Estaciones: %{customdata}<br>" +
                             "<extra></extra>",
               customdata=muestras_anio['Estaciones']),
        secondary_y=False
    )
    
    # Añadir línea para muestras por estación
    fig.add_trace(
        go.Scatter(x=muestras_anio['Año'], y=muestras_anio['Muestras_por_Estacion'].round(1),
                  name="Muestras/Estación", line=dict(color='firebrick', width=2),
                  mode='lines+markers',
                  hovertemplate="Año: %{x}<br>" +
                                "Muestras/Estación: %{y:.1f}<br>" +
                                "<extra></extra>"),
        secondary_y=True
    )
    
    # Configurar diseño del gráfico anual
    fig.update_layout(
        title={
            'text': 'Evolución Temporal del Monitoreo de Calidad de Agua',
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        template='plotly_white',
        height=500,
        showlegend=True,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )
    
    fig.update_xaxes(title_text="Año", tickangle=0)
    fig.update_yaxes(title_text="Número Total de Muestras", secondary_y=False)
    fig.update_yaxes(title_text="Muestras por Estación", secondary_y=True)
    
    show_plot_safely(fig, 'muestras_por_anio.html')
    
    # Análisis mensual
    nombres_meses = {1:'Enero', 2:'Febrero', 3:'Marzo', 4:'Abril', 5:'Mayo', 6:'Junio', 
                    7:'Julio', 8:'Agosto', 9:'Septiembre', 10:'Octubre', 11:'Noviembre', 12:'Diciembre'}
    
    estaciones_anio = {1:'Verano', 2:'Verano', 3:'Otoño', 4:'Otoño', 5:'Otoño',
                       6:'Invierno', 7:'Invierno', 8:'Invierno', 9:'Primavera',
                       10:'Primavera', 11:'Primavera', 12:'Verano'}
    
    # Agregación mensual de datos
    muestras_mes = df.groupby('Mes').agg({
        'GLS_ESTACION': 'nunique',
        fecha_col: 'count'
    }).rename(columns={'GLS_ESTACION': 'Estaciones', fecha_col: 'Muestras'}).reset_index()
    
    muestras_mes['Nombre_Mes'] = muestras_mes['Mes'].map(nombres_meses)
    muestras_mes['Estacion'] = muestras_mes['Mes'].map(estaciones_anio)
    muestras_mes['Muestras_por_Estacion'] = muestras_mes['Muestras'] / muestras_mes['Estaciones']
    
    # Gráfico mensual
    fig = go.Figure()
    
    # Barras de muestras totales
    fig.add_trace(go.Bar(
        x=muestras_mes['Nombre_Mes'],
        y=muestras_mes['Muestras'],
        name='Total Muestras',
        marker_color=muestras_mes['Estacion'].map({
            'Verano': '#ff7f0e',    # Naranja
            'Otoño': '#2ca02c',     # Verde
            'Invierno': '#1f77b4',   # Azul
            'Primavera': '#d62728'   # Rojo
        }),
        text=muestras_mes['Muestras'],
        textposition='auto',
        hovertemplate="Mes: %{x}<br>" +
                      "Muestras: %{y}<br>" +
                      "Estaciones: %{customdata}<br>" +
                      "<extra></extra>",
        customdata=muestras_mes['Estaciones']
    ))
    
    # Línea de muestras por estación
    fig.add_trace(go.Scatter(
        x=muestras_mes['Nombre_Mes'],
        y=muestras_mes['Muestras_por_Estacion'],
        name='Muestras/Estación',
        mode='lines+markers',
        line=dict(color='black', width=2),
        marker=dict(size=8),
        yaxis='y2'
    ))
    
    # Configurar diseño del gráfico mensual
    fig.update_layout(
        title={
            'text': 'Distribución Mensual de Muestras por Estación del Año',
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        xaxis_title="Mes",
        yaxis_title="Número Total de Muestras",
        yaxis2=dict(
            title="Muestras por Estación",
            overlaying='y',
            side='right'
        ),
        template='plotly_white',
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    
    show_plot_safely(fig, 'muestras_por_mes.html')
    
    # Estadísticas por estación del año
    print("\nEstadísticas por estación del año:")
    estacion_stats = df.groupby('Estacion_Anio').agg({
        'GLS_ESTACION': 'nunique',
        fecha_col: ['count', 'nunique']
    }).round(2)
    estacion_stats.columns = ['Estaciones', 'Total_Muestras', 'Dias_Muestreados']
    estacion_stats['Muestras_por_Estacion'] = (estacion_stats['Total_Muestras'] / 
                                              estacion_stats['Estaciones']).round(2)
    print(estacion_stats)

## 5. 📋 Estadísticas descriptivas de parámetros físico-químicos

Para entender realmente la condición de los cuerpos de agua estudiados, necesitamos profundizar en los parámetros físico-químicos que definen su calidad. Estos parámetros son indicadores clave que nos revelan tanto procesos naturales como posibles alteraciones de origen antropogénico.

En esta sección calculamos las principales estadísticas descriptivas y exploramos la distribución de cada parámetro mediante visualizaciones que facilitan la identificación de patrones y anomalías.

Los boxplots resultan especialmente útiles para este análisis ya que permiten observar simultáneamente:

- Valores típicos (medianas, cuartiles) que nos indican las condiciones normales
- Rangos de variación natural en diferentes escalas temporales y espaciales
- Valores atípicos que podrían representar eventos extraordinarios o errores de medición

Al analizar estos parámetros no solo obtenemos una fotografía del estado actual de los ecosistemas acuáticos, sino que establecemos también la base para identificar umbrales críticos, necesarios para una gestión efectiva del recurso hídrico.

---

En esta sección realizamos un análisis estadístico riguroso de los parámetros físico-químicos que caracterizan la calidad del agua en los cuerpos lacustres estudiados. Este análisis proporciona una comprensión de la variabilidad natural y posibles anomalías en los indicadores ambientales monitoreados.

### Análisis implementados:

- **Medidas de tendencia central**: Evaluación de valores medios, medianas y modas para establecer los niveles típicos de cada parámetro

- **Medidas de dispersión**: Cuantificación de la variabilidad a través de desviación estándar, rango y rango intercuartílico

- **Distribución y asimetría**: Caracterización de la forma de la distribución mediante visualizaciones de tipo boxplot

- **Detección de valores atípicos**: Identificación de observaciones extremas que podrían representar anomalías ambientales o errores de medición

Este análisis descriptivo constituye la base para interpretaciones más avanzadas, permitiendo establecer rangos normales de variación y umbrales críticos para cada parámetro, fundamentales para la gestión de la calidad ambiental.

In [5]:
# Estadísticas y boxplots para parámetros clave
for param in parametros:
    if param in df.columns:
        print(f"\nResumen estadístico de {param}:")
        print(df[param].describe())
        fig = px.box(df, y=param, points='outliers',
                     title=f'Distribución de {param}', template='plotly_white')
        show_plot_safely(fig, f'distribucion_{param}.html')

NameError: name 'parametros' is not defined

## 6. 📉 Comparación de parámetros por estación y región

Una de las cosas que más me intriga como analista ambiental es cómo varían los parámetros entre diferentes ubicaciones. Esta variabilidad espacial puede decirnos muchísimo sobre las presiones locales y las particularidades de cada ecosistema.

Recuerdo un proyecto en la cuenca del río Bío-Bío donde estaba analizando datos de calidad de agua. Los valores de pH variaban sorprendentemente entre estaciones ubicadas a pocos kilómetros de distancia. Lo que parecía una anomalía resultó ser el efecto de diferentes sustratos geológicos — algo que no habríamos detectado sin un análisis comparativo entre estaciones.

Al comparar parámetros entre ubicaciones, me gusta plantearme algunas preguntas:

- ¿Existen "perfiles" característicos para ciertos tipos de lagos o regiones? Una vez encontré que todos los lagos de origen volcánico tenían un patrón de mineralización distintivo.

- ¿Las diferencias observadas son estadísticamente significativas o podrían deberse al azar? Cuando trabajaba en consultora, a menudo teníamos que defender nuestras conclusiones ante autoridades regulatorias, lo que me enseñó a ser riguroso con la significancia estadística.

- ¿Hay estaciones que consistentemente muestran valores extremos en múltiples parámetros? Estas "señales de alarma" son especialmente útiles para priorizar esfuerzos de mitigación.

Los boxplots agrupados que utilizo a continuación son una de mis herramientas favoritas para este tipo de análisis. Permiten comparar no solo las medianas sino también la dispersión y la presencia de valores atípicos entre diferentes grupos espaciales.

---

Esta sección aborda la variabilidad espacial de los parámetros físico-químicos, analizando cómo difieren los indicadores de calidad del agua entre distintas ubicaciones geográficas a nivel nominal (estaciones y regiones).

### Enfoques analíticos implementados:

- **Análisis de varianza implícito**: Evaluación visual de las diferencias en la distribución de parámetros entre grupos espaciales

- **Comparación multi-escala**: Contraste entre patrones observables a nivel de estación específica y a nivel regional

- **Heterogeneidad espacial**: Cuantificación del grado de variabilidad que puede atribuirse a factores geográficos

- **Identificación de anomalías localizadas**: Detección de estaciones o regiones con comportamientos atípicos en determinados parámetros

Este análisis comparativo permite identificar zonas que requieren atención prioritaria debido a condiciones extremas, así como comprender los factores geográficos que podrían influir en la calidad del agua. Las visualizaciones mediante boxplots facilitan la comparación intuitiva de distribuciones complejas entre múltiples ubicaciones.

In [ ]:
# Boxplots por estación y región para parámetros seleccionados
for param in parametros:
    if param in df.columns and 'GLS_ESTACION' in df.columns:
        fig = px.box(df, x='GLS_ESTACION', y=param, points='outliers',
                     title=f'{param} por Estación (Top 20)',
                     category_orders={'GLS_ESTACION': estacion_counts['Estación'].tolist()[:20]},
                     template='plotly_white')
        fig.update_xaxes(tickangle=45)
        show_plot_safely(fig, f'boxplot_estacion_{param}.html')
    
    if param in df.columns and 'region' in df.columns:
        fig = px.box(df, x='region', y=param, points='outliers',
                     title=f'{param} por Región', template='plotly_white')
        show_plot_safely(fig, f'boxplot_region_{param}.html')

## 7. ⚠️ Identificación de valores críticos y focos de atención

Esta sección se centra en la identificación y análisis de zonas que presentan condiciones potencialmente problemáticas para los ecosistemas acuáticos. El análisis se fundamenta en criterios técnicos establecidos internacionalmente para parámetros clave de calidad del agua.

### Parámetros evaluados:

**pH (rango óptimo 6-9)**
- Criterio basado en estándares internacionales para la protección de vida acuática
- Análisis de frecuencia de valores fuera de rango por estación
- Identificación de patrones espaciales en valores críticos

**Oxígeno disuelto (nivel crítico < 5 mg/L)**
- Umbral establecido según requerimientos biológicos de especies acuáticas
- Evaluación de frecuencia y duración de eventos hipoxícos
- Correlación con otros parámetros físico-químicos

La identificación sistemática de valores críticos permite:
- Priorizar zonas para intervenciones de gestión ambiental
- Evaluar la efectividad de medidas de control implementadas
- Establecer líneas base para futuros programas de monitoreo

In [ ]:
# Proporción de valores críticos de pH y oxígeno disuelto
if 'ph' in df.columns:
    df['ph_estado'] = df['ph'].apply(lambda x: 'Óptimo (6-9)' if 6 <= x <= 9 else 'Crítico')
    ph_critico = df.groupby('GLS_ESTACION')['ph_estado'].value_counts(normalize=True).unstack().fillna(0)
    ph_critico = ph_critico.sort_values('Crítico', ascending=False)
    fig = px.bar(ph_critico.head(15), y='Crítico',
                 title='Top 15 Estaciones con Mayor Proporción de pH Crítico',
                 labels={'GLS_ESTACION':'Estación','Crítico':'Proporción Crítica'},
                 template='plotly_white')
    show_plot_safely(fig, 'ph_critico.html')

if 'oxigeno_disuelto_mg_l' in df.columns:
    df['ox_estado'] = df['oxigeno_disuelto_mg_l'].apply(lambda x: 'Bajo (<5 mg/L)' if x < 5 else 'Adecuado (≥5 mg/L)')
    ox_critico = df.groupby('GLS_ESTACION')['ox_estado'].value_counts(normalize=True).unstack().fillna(0)
    ox_critico = ox_critico.sort_values('Bajo (<5 mg/L)', ascending=False)
    fig = px.bar(ox_critico.head(15), y='Bajo (<5 mg/L)',
                 title='Top 15 Estaciones con Mayor Proporción de Oxígeno Disuelto Bajo',
                 labels={'GLS_ESTACION':'Estación','Bajo (<5 mg/L)':'Proporción Crítica'},
                 template='plotly_white')
    show_plot_safely(fig, 'oxigeno_critico.html')

## 12. 📚 Conclusiones y Recomendaciones

El análisis de calidad del agua en cuerpos lacustres de Chile ha generado hallazgos significativos que fundamentan recomendaciones técnicas para la gestión de recursos hídricos.

### 📈 Hallazgos clave

1. **Distribución espacial del monitoreo**
   - Concentración del 80% de muestras en 20% de las estaciones
   - Identificación de brechas significativas en la cobertura territorial
   - Patrones regionales de muestreo que requieren optimización

2. **Dinámica temporal**
   - Variaciones estacionales significativas en parámetros clave
   - Tendencias interanuales que sugieren cambios sistemáticos
   - Identificación de periodos críticos que requieren mayor frecuencia de monitoreo

3. **Parámetros críticos**
   - Estaciones con frecuencia elevada de pH fuera del rango óptimo
   - Zonas recurrentes de bajo oxígeno disuelto
   - Correlaciones significativas entre parámetros que sugieren procesos biogequímicos

### ⚙️ Recomendaciones técnicas

1. **Optimización del monitoreo**
   - Redistribución espacial de puntos de muestreo
   - Implementación de coordenadas GPS precisas
   - Estandarización de protocolos de muestreo y análisis

2. **Mejoras metodológicas**
   - Inclusión de parámetros biológicos complementarios
   - Establecimiento de frecuencias mínimas de muestreo por estación
   - Implementación de controles de calidad in situ

3. **Gestión de datos**
   - Desarrollo de sistema de alerta temprana para valores críticos
   - Implementación de plataforma de visualización en tiempo real
   - Integración con otros conjuntos de datos ambientales

Las conclusiones derivadas proporcionan una base sólida para la toma de decisiones en gestión ambiental y el diseño de futuros programas de monitoreo de calidad del agua.

## 9. 🔗 Análisis de Correlaciones entre Parámetros

Una de las partes más reveladoras del análisis de datos ambientales es descubrir cómo se relacionan los diferentes parámetros entre sí. Estas relaciones pueden decirnos mucho sobre los procesos naturales que ocurren en los ecosistemas acuáticos.

En esta sección, quiero explorar preguntas como:

- ¿Qué parámetros tienden a moverse juntos? Por ejemplo, en muchos lagos que he estudiado, la temperatura y el oxígeno disuelto suelen tener una correlación negativa debido a la menor solubilidad del oxígeno en agua caliente.

- ¿Hay correlaciones inesperadas que sugieran procesos específicos de estos cuerpos de agua? Las sorpresas en los datos suelen ser las que generan las hipótesis más interesantes.

- ¿Qué parámetros parecen ser redundantes? Esto es útil para optimizar futuros programas de monitoreo, ahorrando recursos en mediciones que aportan información similar.

Utilizo la correlación de Pearson por su interpretabilidad, complementada con visualizaciones mediante mapas de calor que permiten identificar patrones complejos de forma intuitiva. Me centro especialmente en correlaciones fuertes (|r| > 0.5) que sugieren relaciones significativas entre parámetros.

Este análisis de correlaciones proporciona insights valiosos sobre la estructura multivariada de los datos, permitiendo:
- Identificar redundancias en la información capturada por diferentes parámetros
- Detectar posibles mecanismos causales que conectan distintos aspectos de la calidad del agua
- Optimizar futuros protocolos de monitoreo, enfocando recursos en parámetros independientes que maximicen la información capturada

In [ ]:
# Calcular y visualizar matriz de correlaciones
if df is not None:
    # Seleccionar solo columnas numéricas de parámetros físico-químicos
    param_numericos = df[parametros].select_dtypes(include=['float64', 'int64']).columns
    if len(param_numericos) > 1:
        # Calcular correlaciones
        corr_matrix = df[param_numericos].corr()
        
        # Crear mapa de calor interactivo
        fig = px.imshow(corr_matrix,
                       title='Matriz de Correlaciones entre Parámetros',
                       template='plotly_white',
                       color_continuous_scale='RdBu',
                       aspect='auto')
        fig.update_traces(text=corr_matrix.round(2), texttemplate='%{text}')
        show_plot_safely(fig, 'correlaciones.html')
        
        # Identificar correlaciones más fuertes
        print('\nCorrelaciones más significativas (|r| > 0.5):')
        for i in range(len(param_numericos)):
            for j in range(i+1, len(param_numericos)):
                corr = corr_matrix.iloc[i,j]
                if abs(corr) > 0.5:
                    print(f'{param_numericos[i]} vs {param_numericos[j]}: {corr:.2f}')

## 10. 📈 Análisis de Tendencias Temporales

El análisis de tendencias temporales es fundamental para entender cómo evolucionan los ecosistemas acuáticos. Cuando trabajaba en la evaluación de impacto ambiental de una central hidroeléctrica, descubrimos que algunas tendencias que parecían alarmantes en el corto plazo eran en realidad parte de ciclos naturales más amplios — algo que solo pudimos detectar porque contábamos con una serie temporal suficientemente larga.

En esta sección me interesa especialmente:

- Detectar posibles tendencias de deterioro o mejora en la calidad del agua que podrían no ser evidentes en análisis puntuales

- Diferenciar entre fluctuaciones estacionales (que son parte del comportamiento normal de los ecosistemas) y cambios direccionales que podrían indicar problemas emergentes

- Evaluar si las intervenciones de gestión ambiental implementadas en ciertos periodos han tenido efectos medibles en la calidad del agua

Para este análisis, trabajo con promedios mensuales para reducir el ruido en los datos, y aplico técnicas de regresión local (LOWESS) que permiten capturar relaciones no lineales sin imponer un modelo estadístico rígido. También analizo la variabilidad estacional, que en Chile puede ser muy marcada debido a las diferencias pluviométricas entre estaciones.

In [ ]:
# Análisis de tendencias temporales por parámetro
if df is not None and fecha_col:
    for param in parametros:
        if param in df.columns:
            # Calcular promedios mensuales
            df['Año_Mes'] = df[fecha_col].dt.to_period('M')
            promedios_mensuales = df.groupby('Año_Mes')[param].mean().reset_index()
            promedios_mensuales[fecha_col] = promedios_mensuales['Año_Mes'].dt.to_timestamp()
            
            # Graficar serie temporal con línea de tendencia
            fig = px.scatter(promedios_mensuales, x=fecha_col, y=param,
                          title=f'Tendencia Temporal de {param}',
                          template='plotly_white',
                          trendline='lowess')
            fig.update_traces(marker=dict(size=6))
            show_plot_safely(fig, f'tendencia_{param}.html')
            
            # Calcular estadísticas por estación del año
            df['Estacion'] = df[fecha_col].dt.quarter.map({1:'Verano', 2:'Otoño', 3:'Invierno', 4:'Primavera'})
            stats_estacional = df.groupby('Estacion')[param].agg(['mean', 'std']).round(2)
            print(f'\nEstadísticas estacionales para {param}:')
            print(stats_estacional)

## 11. 🛠️ Validación de Datos y Control de Calidad

Esta sección implementa un riguroso proceso de validación de datos para evaluar la fiabilidad del conjunto de datos y detectar posibles anomalías que podrían comprometer la integridad de los análisis y conclusiones.

### Protocolos de validación implementados:

- **Análisis de valores atípicos robustos**: Aplicación del método de rango intercuartílico (IQR) con factor de escala 3 para la detección de valores extremos, equilibrando sensibilidad y especificidad

- **Establecimiento de rangos teóricos esperados**: Definición de umbrales de validación basados en principios fisico-químicos y referencias bibliográficas para cada parámetro

- **Evaluación espacial de anomalías**: Identificación de estaciones con frecuencia inusualmente alta de valores atípicos, que podrían indicar problemas sistemáticos de medición o condiciones ambientales particulares

- **Visualización especializada**: Implementación de gráficos de caja optimizados para destacar la distribución de valores atípicos

Este proceso de validación es fundamental para:

- Distinguir entre variabilidad natural y errores de medición o registro
- Evaluar la necesidad de técnicas estadísticas robustas en análisis posteriores
- Formular recomendaciones para mejorar los protocolos de recolección y procesamiento de datos
- Aumentar la confianza en las conclusiones derivadas del análisis

In [ ]:
# Análisis de valores atípicos y validación
if df is not None:
    for param in parametros:
        if param in df.columns:
            # Calcular estadísticas básicas
            stats = df[param].describe()
            Q1 = stats['25%']
            Q3 = stats['75%']
            IQR = Q3 - Q1
            lower_bound = Q1 - 3 * IQR
            upper_bound = Q3 + 3 * IQR
            
            # Identificar valores atípicos extremos
            outliers = df[(df[param] < lower_bound) | (df[param] > upper_bound)]
            
            print(f'\nValidación de {param}:')
            print(f'Rango esperado: [{lower_bound:.2f}, {upper_bound:.2f}]')
            print(f'Valores atípicos detectados: {len(outliers)}')
            
            if len(outliers) > 0:
                print('\nEstaciones con mayor frecuencia de valores atípicos:')
                print(outliers['GLS_ESTACION'].value_counts().head())
                
                # Visualizar distribución con valores atípicos marcados
                fig = go.Figure()
                fig.add_trace(go.Box(y=df[param], name=param,
                                   boxpoints='outliers',
                                   marker_color='blue',
                                   boxmean=True))
                fig.update_layout(title=f'Distribución y Valores Atípicos de {param}',
                                template='plotly_white')
                show_plot_safely(fig, f'atipicos_{param}.html')

## 13. 📚 Referencias y Metodología

### Fundamentos metodológicos

El análisis implementado en este notebook se basa en metodologías estandarizadas para la evaluación de calidad de agua y principios de ciencia de datos, incluyendo:

- **Criterios de calidad de agua**: Los umbrales utilizados para la clasificación de valores críticos se basan en estándares internacionales, como los lineamientos de la Organización Mundial de la Salud (OMS) y la Agencia de Protección Ambiental de Estados Unidos (EPA).

- **Métodos estadísticos**: Se aplicaron técnicas robustas para el análisis descriptivo, detección de valores atípicos y análisis de correlaciones, siguiendo principios establecidos en la literatura estadística.

- **Visualización de datos**: Las representaciones gráficas implementadas siguen principios de comunicación visual efectiva, maximizando la relación señal-ruido y facilitando la interpretación intuitiva de patrones complejos.

### Referencias bibliográficas

1. Chapman, D. (1996). Water Quality Assessments: A Guide to the Use of Biota, Sediments and Water in Environmental Monitoring. UNESCO/WHO/UNEP.

2. Wetzel, R. G. (2001). Limnology: Lake and River Ecosystems (3rd ed.). Academic Press.

3. World Health Organization. (2017). Guidelines for drinking-water quality (4th ed.). WHO.

4. Reynolds, C. S. (2006). The Ecology of Phytoplankton. Cambridge University Press.

5. Tufte, E. R. (2001). The Visual Display of Quantitative Information (2nd ed.). Graphics Press.

6. Wickham, H. (2016). ggplot2: Elegant Graphics for Data Analysis (2nd ed.). Springer.

7. Dirección General de Aguas. (2023). Norma Secundaria de Calidad Ambiental para la Protección de las Aguas Continentales Superficiales. Ministerio de Medio Ambiente, Chile.

### Limitaciones del análisis

Es importante reconocer las siguientes limitaciones que deben considerarse al interpretar los resultados:

- La ausencia de coordenadas geográficas precisas limita la implementación de análisis espaciales avanzados como kriging o modelado geoestadístico.

- La heterogeneidad temporal del muestreo puede introducir sesgos en la comparación de tendencias entre estaciones con diferentes frecuencias de monitoreo.

- No se dispone de información sobre factores contextuales como usos de suelo circundante, actividades económicas cercanas o eventos climáticos extremos que podrían explicar algunas de las variaciones observadas.

- Los parámetros fisico-químicos analizados, aunque informativos, no capturan la totalidad de aspectos relevantes para la salud ecosistémica, como contaminantes emergentes o indicadores biológicos.